# HW6

In this assignment you'll be building a "dashboard" interface for the buildings dataset which provides the user with a linked view of this dataset.  The purpose of this assignment is to get some "hands-on" practice linking different facets of a dataset in a dashboard to provide the user a way to interactively browse their data.

**Build a dashboard for the buildings data.**

**Left component:**

Grid heat map; Columns are County; Rows are the governmental department (Agency Name) (note, the agency names might overlap with your plot -- there are ways around this or you can leave as is); Values are mean of total square footage for that set of criteria

**Right component:**

Barplot/histogram; x is the year; y is total square footage acquired that year

These two should be linked so that you can select cells and that will update the square footage plot.

**Things to think about:**

Can you keep the x and y ranges static on the bar plot?

Can you change the style?

**In your write-up please address:**

Any data transformations or rescalings you did for your plot

How you dealt with NaN's (empty entries) in your dataset

Any aesthetic choices you made (colors, layout, plot size, label size) and why, and what you'd like to experiment with if you had more time.

You don't have to use bqplot (you can use other libraries in Python if you'd like), but bqplot is probably the easiest.
20 points for grid heatmap, 20 points for line plot, and 20 points for accurate linkage between the two.  40 points for writeup.

In [1]:
import pandas as pd
import numpy as np
import ipywidgets
import bqplot

In [2]:
buildings = pd.read_csv('https://uiuc-ischool-dataviz.github.io/is445_AOUAOG_fall2021/week02/data/building_inventory.csv',
                        na_values={'Square Footage':0,
                                  'Year Acquired':0,
                                  'Year Constructed':0})
buildings.head()

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975.0,1975.0,144.0,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004.0,2004.0,144.0,1,1,0,Unusual,Unusual,Not provided


In [3]:
grouped = buildings.groupby('Agency Name').County.value_counts().sort_index()
# print(grouped)
data = grouped.unstack()
data.fillna(value=0, inplace=True) # Replacing NaN values by zero
data.head()

County,Adams,Alexander,Bond,Boone,Brown,Bureau,Calhoun,Carroll,Cass,Champaign,...,Wabash,Warren,Washington,Wayne,White,Whiteside,Will,Williamson,Winnebago,Woodford
Agency Name,,,,,,,,,,,,,,,,,,,,,
Appellate Court / Fifth District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appellate Court / Fourth District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appellate Court / Second District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appellate Court / Third District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Chicago State University,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
x = data.columns
y = data.index
row_agency = y.tolist()
column_county = x.tolist()

## Part1. Histogram

In [5]:
# 1. Data -- hard coded to start
i,j = 33, 9

# 2. Scales -- linear for a histogram of numerical data
x_sch = bqplot.LinearScale()
y_sch = bqplot.LinearScale()

# 3. Axis
x_axh = bqplot.Axis(scale=x_sch, label='Year')
y_axh = bqplot.Axis(scale=y_sch, orientation='vertical', label_offset = "50px", label='Total Square Footage Acquired')

# 4. Marks -- histogram, passing "sample" data attribute
hist_sample = buildings.loc[(buildings['County']==column_county[j]) & 
                            (buildings['Agency Name']==row_agency[i])].groupby("Year Acquired")["Square Footage"].sum()
hist = bqplot.Bars(x=hist_sample.index, y=hist_sample.values,
                  scales={'x':x_sch, 'y':y_sch})

# 6. figure!
figh = bqplot.Figure(marks=[hist], axes=[x_axh, y_axh])
# figh

## Part2. Heat Map

In [6]:
# 1. Data

In [7]:
# 2. Scales
col_sc = bqplot.ColorScale(scheme="plasma") 
x_sc = bqplot.OrdinalScale() # treating bins like bins of categorical data
y_sc = bqplot.OrdinalScale()

In [8]:
# 3. Axis -- color & for x/y
col_ax = bqplot.ColorAxis(scale=col_sc, orientation='vertical', side='right')
ax_x = bqplot.Axis(scale=x_sc, label='County', tick_rotate = 90, tick_style = {'font-size': 8})
ax_y = bqplot.Axis(scale=y_sc, orientation='vertical', label='Agency Name', label_offset = "150px", tick_style = {'font-size': 8})

In [9]:
# 4. Mark -- heatmap
heat_map = bqplot.GridHeatMap(color=data,
                              row=row_agency, column=column_county,
                              scales={'color':col_sc, 'row':y_sc, 'column':x_sc}, 
                              interactions={'click':'select'}, 
                              anchor_style={'fill':'blue'})

In [10]:
# 5. Add interactions
mySelectedLabel = ipywidgets.Label()
def on_selected(change):
    if len(change['owner'].selected) == 1: # only allow user to select 1 bin
        i,j = change['owner'].selected[0] # grab row/column ID
        selected_df = buildings.loc[(buildings['County']==column_county[j]) & (buildings['Agency Name']==row_agency[i])]
        mean_sf = round(selected_df["Square Footage"].mean(), 4)
        mySelectedLabel.value = 'Mean of total square footage is = ' + str(mean_sf)
        hist.x = selected_df.groupby("Year Acquired")["Square Footage"].sum().index
        hist.y = selected_df.groupby("Year Acquired")["Square Footage"].sum().values
# add the .observe link
heat_map.observe(on_selected, 'selected')

## Part3. Combine and show figure

In [11]:
fig = bqplot.Figure(marks=[heat_map], axes=[col_ax, ax_x, ax_y], fig_margin= dict(top=50, bottom=40, left=200, right=60))
fig.layout.min_width='1500px'
figh.layout.min_width = '600px'
figures = ipywidgets.HBox([fig, figh])
myDashboard = ipywidgets.VBox([mySelectedLabel,figures])
myDashboard

## Part4. Write-ups

**Q1. Any data transformations or rescalings you did for your plot?**

1. While reading the buildings dataset, I transformed all the "nan" values to "0".

2. In order to plot the heat map with columns as counties and rows as agency names, I grouped the buildings data frame by "Agency Name" and then got the row counts (number of buildings) for each county.

3. I unstacked the grouped results into a pivot table (data) and filled the nan cells by 0 as the basis of the heat map. Since both "County" and "Agency Name" have categorical values, I selected OrdinalScale for both the x and y axes.

4. In order to plot the histogram/bar graph for the selected cell in the heat map, I set the scales for x and y axes as LinearScale since both "Year" and "Total Square Footage Acquired" have numerical values. To generate the total square footage data for each selected heat map cell, I used .loc to filter the original buildings dataset to keep only the rows with the selected county and agency names. Then, I calculated the mean square footage under the condition as the selected label value to be printed when the users make selections on the heatmap.

5. Before passing the data to the histogram/bar plot, I grouped the selected data frame by the "Year Acquired" and calculated the "Square Footage" sum as the total square footage. And I assigned the index of the sum grouped results as the x-axis of the histogram and the values as the y axis of the histogram.


**Q2. How you dealt with NaN's (empty entries) in your dataset?**

I transformed all the NaNs in the original building inventory dataset as zeros while reading the CSV datafile as a pandas data frame. Moreover, to generate the pivot table before plotting the heatmap, I also got Nans in my unstacked grouped results, and I filled all NaNs with zeros in my dataset.


**Q3. Any aesthetic choices you made (colors, layout, plot size, label size) and why, and what you'd like to experiment with if you had more time.**

1. I selected the color scheme as "plasma" since it has more contrasts than a single color with a white background. From the above heatmap plotting, I can quickly identify some special cells with bright yellow or orange colors, which alerts me with some exceptional cases with the outstanding number of buildings built for a certain agency in a certain county.

2. After plotting my first heatmap, I noticed there are overlaps in both x and y axes and in the y-label and the y-tickers. In order to solve this problem, I first rotated the x ticks by 90 degrees to show them vertically and sized down the font of x and y ticks to 8 while identifying the axes in plotting. Then, I set the label_offset value of the y-axis as "150px" to make the y-label "Agency Name" stay further from the y axis.

3. In the final step of combining the heatmap and histogram side by side, I set the figure margins as (top=50, bottom=40, left=200, right=60) to make everything visible on the canvas. And since there are too many grids in the heatmap, to make the cells visible and easy to be selected, I set the minimum width of the heatmap layout to 1500px and the minimum width of the histogram layout to 600px.

Things I would like to experiment with:

1. Instead of showing the heatmap and the histogram side by side, I would like to check whether I can only temporarily show the histogram above the selected cell.

2. I think the "County" might be too detailed in analyzing the distribution of buildings. Maybe "State" would be a better unit in analyzing. And I can perhaps try a US map shape heatmap to show the distributions of buildings all over the country.

3. Instead of plotting the histogram to show the total square footage acquired for each year, I can also plot the scatter plot to show the square footage acquired/constructed each year to discover the pattern/trend.
